<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/8.Machine_Learning/03_features_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Transformación de variables



# Reescalado de variables

Reescalar consiste en ajustar los datos a una escala determinada, con unos límites superior e inferior. Una de escalas más usadas es la minmax, en la que se escalan los valores mediante la siguiente expresión.






\begin{equation}
Z = \frac{x - min(x)}{max(x) - min(x)}
\end{equation}



En Python podemos aplicar esta escala con los siguiente pasos.

In [ ]:
from sklearn import preprocessing
import numpy as np



Creamos un conjunto de datos

In [ ]:

x = np.array([[-500.5], 
              [-100.1], 
              [0], 
              [100.1], 
              [900.9]])



Creamos una objeto MinMaxScaler

In [ ]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))



Aplicamos la escala

In [ ]:
x_scale = minmax_scale.fit_transform(x)



Vemos la transformación

In [ ]:
x_scale



# Standarización de variables

La standarización es otro método de escalado. Consiste en reescalar una variable para que tenga media 0 y una varianza igual a 1.

Vemos como aplicarlo en Python.



In [ ]:
from sklearn import preprocessing
import numpy as np



Creamos un conjutno de datos

In [ ]:
x = np.array([[-500.5], 
              [-100.1], 
              [0], 
              [100.1], 
              [900.9]])



Creamos el objeto para escalar

In [ ]:
scaler = preprocessing.StandardScaler()



Lo aplicamos

In [ ]:
standardized = scaler.fit_transform(x)



Vemos la transformación

In [ ]:
standardized



# Discretización de variables

Consiste en el proceso de asignar los datos a una serie de valores discretos.

In [ ]:
from sklearn.preprocessing import Binarizer
import numpy as np

In [ ]:
age = np.array([[6], 
                [12], 
                [20], 
                [36], 
                [65]])



Tenemos dos opciones en Python, usar un Binarizer, que asigna un valor u otro en función de un límite, o crear unos bins.



Creamos el Binarizer indicando el límite de edad de 18

In [ ]:
binarizer = Binarizer(threshold=18)



Aplicamos la transformación

In [ ]:
binarizer.fit_transform(age)



Creamos una serir de bins a los que asignar los valores

In [ ]:
np.digitize(age, bins=[20,30,64])



# Reshape

El proceso de reshape consiste en dar una nueva forma a la estructura de los datos, sin modificarlos.





Por ejemplo, dada la matriz 4x3

In [ ]:
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [10, 11, 12]])
matrix



Podemos transformar los datos por ejemplo en una matriz de 2x6

In [ ]:
matrix.reshape(2, 6)



# One-Hot Encoder

El proceso de One-hot encoding consiste en la transformación de variables categóricas en una nueva forma que se puede alimentar a los algoritmos de modo que de lugar a una mejora en las predicciones.


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler




Creamos unos datos categóricos de ejemplo

In [ ]:
y = [('Texas', 'Florida'), 
    ('California', 'Alabama'), 
    ('Texas', 'Florida'), 
    ('Delware', 'Florida'), 
    ('Texas', 'Alabama')]
y



Donde tenemos 5 grupos con 5 elementos únicos. Podemos transformar esta información en un array de 5x5 en cada elemento esté representado por un 1 ó 0, indicando la existencia o no del mismo.



Creamos un objeto MultiLabelBinarizer

In [ ]:
one_hot = MultiLabelBinarizer()




Realizamos la transformación

In [ ]:
one_hot.fit_transform(y)



Vemos cúal es el mapeo de datos

In [ ]:
one_hot.classes_



# Pipelines

Un pipeline consiste en una serie encadenada de transformaciones sobre los datos ( por ejemplo normalizar númericos, tratar NaNs, etc...) junto con un estimador (algoritmo). Este proceso permite realizar todos los procesos en bloque.



Cargamos una dataset de ejemplo con datos sobre cancer

In [ ]:
dataset = datasets.load_breast_cancer()

In [ ]:
dataset.DESCR

In [ ]:
dataset.data.shape



Create X from the dataset's features

In [ ]:
X = dataset.data



Create y from the dataset's output

In [ ]:
y = dataset.target



A continuación creamos una serie de transformaciones sobre los datos:

    - escalado
    - reducción de dimensiones mediante un PCA

Y por último aplicamos un estimador, en este caso una LogisticRegression.




Creamos el objeto de escalado

In [ ]:
sc = StandardScaler()



Creamos el objeto PCA

In [ ]:
pca = decomposition.PCA()



Creamos el estimador logistic regression

In [ ]:
logistic = linear_model.LogisticRegression(solver='liblinear')



Por último enlazamos en un pipeline los 3 pasos.

In [ ]:
pipe = Pipeline(steps=[('sc', sc), 
                       ('pca', pca), 
                       ('logistic', logistic)])



Adicionalmente creamos una rejilla de parámetros que nos permita obtener el modelo con mejores métricas.



Creamos una lista de componentes para el PCA

In [ ]:
n_components = list(range(1,X.shape[1]+1,1))
n_components = [2,3,4,5,10]



Creamos una lista de componentes para el parámetro de regularización de la regresión.

In [ ]:
C = np.logspace(-4, 4, 50)



Creamos una lista de componentes para la penalización de la regresión.

In [ ]:
#penalty = ['l2']
penalty = ['l1','l2']



Por último generamos un diccionario con todos estos parámetros.

In [ ]:
parameters = dict(pca__n_components=n_components, 
                  logistic__C=C,
                  logistic__penalty=penalty)

In [ ]:
parameters



Uniendo estos elementos en un objeto GridSearchCV podemos hacer una búsqueda de los parámetros que optimizan el resultado y evaluarlo mediante validación cruzada.



Creamos un objeto gridsearch

In [ ]:
clf = GridSearchCV(pipe, parameters, n_jobs=-1)




Ajustamos la rejilla

In [ ]:
clf.fit(X, y)



Vemos los mejores parámetros

In [ ]:
print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
print('Best C:', clf.best_estimator_.get_params()['logistic__C'])
print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])



Ajustamos el gridsearch usando validación cruzada con k=3

In [ ]:
cross_val_score(clf, X, y)



# Ejercicios 

Dados los siguientes datos, la representación gráfica de las variables numéricas y la accuracy de un KNN.

In [ ]:
import pandas as pd

In [ ]:
X_train = pd.read_csv('https://github.com/al34n1x/DataScience/raw/master/data/X_train.csv',squeeze=True, delimiter=',')
Y_train = pd.read_csv('https://github.com/al34n1x/DataScience/raw/master/data/Y_train.csv',squeeze=True, delimiter=',')
X_test = pd.read_csv('https://github.com/al34n1x/DataScience/raw/master/data/X_test.csv',squeeze=True, delimiter=',')
Y_test = pd.read_csv('https://github.com/al34n1x/DataScience/raw/master/data/Y_test.csv',squeeze=True, delimiter=',')

In [ ]:
print (X_train.head())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline



Representamos las variables numéricas

In [ ]:
X_train[X_train.dtypes[(X_train.dtypes=="float64")|(X_train.dtypes=="int64")]
                        .index.values].hist(figsize=[11,11])



Ajustamos a un modelo KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 
                   'Loan_Amount_Term', 'Credit_History']],Y_train)



Obtenemos el valor de accuracy

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,knn.predict(X_test[['ApplicantIncome', 'CoapplicantIncome',
                             'LoanAmount', 'Loan_Amount_Term', 'Credit_History']]))



- a. Realice un escalado min/max sobre las variables numéricas y represente su distribución.
- b. Estudie como afecta al resultado del clasificador la aplicación de un escalado min/max.



Realice un escalado min/max sobre las variables numéricas y represente su distribución.

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui



Estudie como afecta al resultado del clasificador la aplicación de un escalado min/max.

In [ ]:
# Respuesta aqui